<a href="https://colab.research.google.com/github/An-JIeun/studynotes/blob/main/GPT%20Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT Fine Tuning
---
- 원본 작성자 : AISchool ( https://www.udemy.com/course/llm-part-1-llama-2-fine-tuning/?referralCode=32804C68FEF005E82BCF )
- 최종수정일 : 2024-01-31

## 0. 환경 세팅
---

GPT는 LLaMA와 달리, OpenAI 쪽 서버에서 학습을 진행한다. 따라서 많은 컴퓨터 리소스 없이도 파인 튜닝을 시도해볼 수 있다.

In [ ]:
%pip install openai
%pip install python-dotenv # .env 환경을 사용하겠다

코드 작성에 앞서, GPT는 비용이 부과된다. 현시점 기준 fine-tuning에 드는 비용은 다음과 같다. 

- gpt-3.5-turbo : $0.0080 / 1K tokens

- davinci : 

- babbage : 

비용은 업데이트 될 수 있으므로 OpenAI의 가격 정책을 확인해야 한다. 가격 뿐만 아니라, 현재 코드도 어떻게 수정될 지 모르니 문서를 꼭 읽어보자. 대략적으로 어느 정도 가격이 드는지 확인하고자 한다면 아래 링크에 접속해서 가격을 확인해 보자.

[fine-tuning 예상 비용 계산하기]()

In [ ]:
import os
import dotenv
dotenv.load_dotenv()

# 코랩 환경에서 실행하고 있다면 os.getenv("OPENAI_API_KEY") 를 지우고 본인의 api key를 입력하면 된다  
API_KEY = os.getenv("OPENAI_API_KEY") # .env 파일에 있는 OPENAI_API_KEY를 가져온다

import openai
openai.api_key = API_KEY

## 1. 학습 데이터 넣어 주기
---

OpenAI에서의 Fine tuning은 프롬프트를 학습하는 수준이다. 프롬프트에 GPT의 성격, 질문에 대한 예시 답안들을 넣어서 학습시킨다. 최소 학습 프롬프트는 10개이다. 아래에 프롬프트 템플릿과 프롬프트 생성 함수를 만들었으니, 자신이 바라는 gpt의 모습을 생각하며 학습 데이터를 만들어보자.

```
{"messages": [
    {"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, # 시스템 - 자신이 바라는 GPT의 성격을 작성한다
    {"role": "user", "content": "What's the capital of France?"},  # 유저 - 예시 질의
    {"role": "assistant", "content": "Paris, as if everyone doesn't know that already."} # 모델 - 예시 답안
    ]
}

```

In [ ]:
def make_prompt(personality, question, answer):
    prompt = { "message" : [
        {"role":"system", "content" : personality},
        {"role" : "user", "content" : question},
        {"role" : "system", "content" : answer}
    ]
              }
    return prompt

[여기]()에 들어가면 내가 만든 예시 프롬프트 30개가 있다. 

나는 우타프리에 나온 카뮤 라는 캐릭터를 모티브로 프롬프트를 작성했다. 
아래 짤방을 보면 알 수 있듯이 귀족 캐릭터이고 귀족같은 성격이다. 아마도 이 데이터로 학습시키면 말 끝마다 oh my... 이러는 gpt를 구경할 수 있을 것이다.
![]()

학습시킬 프롬프트는 openai에 등록해야 한다. 아래 코드를 실행시키면 파일을 등록하고 ID를 부여받는다.

In [ ]:
file_path = ""
openai.File.create(
  file=open(file_path, "rb"),
  purpose='fine-tune'
)

업로드한 파일 ID를 확인하는 코드이다. 그동안 등록해놨던 파일들에 대한 ID가 쭉 나온다. 이 문서를 보고 있는 사람 대부분은 등록한 파일이 한 개 뿐일테니, 첫 번쨰 파일ID를 변수에 넣어주도록 한다.

In [ ]:
print(openai.File.list())
file_id = openai.File.list()[0]

## 2. Fine-Tuning 작업 등록하기
---
아래 코드를 실행하면 OpenAI 서버 상에 파인튜닝 작업이 등록된다. 진행 상황은 직접 [OpenAI 플랫폼]()에 들어가서 확인해야 한다.
OpenAI 플랫폼의 Fine-Tuning 탭에 들어가면, 자신이 등록해둔 작업에 대한 실시간 학습량이나 진행상황을 보여준다.
서버 상황에 따라 소요시간이 달라질 수 있으며, 너무 오래 걸린다 싶으면 그냥 취소하고 다시 돌리는게 낫다고 한다.

In [ ]:
openai.FineTuningJob.create(training_file="여러분의_File_Id", model="gpt-3.5-turbo")

아래는 작업을 취소하는 코드이다. job_id는 작업에 부여된 id인데, 역시 플랫폼 상에서 확인 가능하다.

In [ ]:
job_id = ''
openai.FineTuningJob.cancel("여러분의_job_id")

작업이 완료된 모델은 .. 이것이 바로 당신이 훈련시켜 탄생한 fine-tuned model이다.

In [1]:
model_id = ""

## 3. 모델 사용해보기
---
각자 원하는 성격대로든, 예시 데이터로든 모델을 학습시켰을 것이다. 직접 훈련한 모델을 사용하려면 앞서 알아낸 모델 ID를 model 파라미터에 넣으면 된다.

- 자신이 훈련한 성격으로 사용

In [ ]:
completion = openai.ChatCompletion.create(
  model=model_id,
  messages=[
    {"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."},
    {"role": "user", "content": "What's the capital of France?"}
  ]
)
print(completion.choices[0].message)

물론, 여기서 시스템 값을 달리 주면 다르게 대답할 수도 있다.

In [ ]:
completion = openai.ChatCompletion.create(
  model="여러분의_모델_id",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What's the capital of France?"}
  ]
)
print(completion.choices[0].message)